In [44]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division
import numpy as np
import pandas as pd
import operator

Populating the interactive namespace from numpy and matplotlib


/Users/uditagupta93/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text', 'delete']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [45]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [46]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens_words(text):
    x = re.findall('[a-z]+', text.lower()) 
    #print(x)
    return str(x[0])

In [47]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488665

In [48]:
#spell_errors = open("spell-errors.txt").read()
#len(spell_errors)

In [49]:
delete = [':','?',';',',','>']

def load_spell_errors(filename, sep=':'):
    spell_errors_dict = {}
    """Return a Counter initialized from key-value pairs, 
    one on each line of filename."""
    for line in open(filename):
        x = line.split(sep)
        key = x[0]
        if key not in delete:
            key = tokens_words(key)
            val = x[1:]
            val = val[0].split(',')
            #print(val)
            for i in range(len(val)):

                val[i] = val[i].strip()
                val[i] = tokens_words(val[i])
                #print(i)


            spell_errors_dict[key]=val

    return spell_errors_dict


spell_errors_dict = load_spell_errors('spell-errors.txt')
print(len(spell_errors_dict))
#print(spell_errors_dict)

7506


In [50]:
WORDS = tokens(TEXT)
counts_sum = len(WORDS)

In [51]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [52]:
def load_counts(filename, sep='\t'):
    """Return a Counter initialized from key-value pairs, 
    one on each line of filename."""
    C = Counter()
    for line in open(filename):
        key, count = line.split(sep)
        C[key] = int(count)
    return C


COUNTS1 = load_counts('count_1w.txt')


In [53]:
print(COUNTS1.most_common(10))


[('the', 23135851162), ('of', 13151942776), ('and', 12997637966), ('to', 12136980858), ('a', 9081174698), ('in', 8469404971), ('for', 5933321709), ('is', 4705743816), ('on', 3750423199), ('that', 3400031103)]


In [54]:
COUNTS1_dict = dict(COUNTS1)

In [55]:
# get the overall corpus count
counts1_sum = 0
for val in COUNTS1_dict.values():
    counts1_sum += val
    
print(counts1_sum)

588124220187


### Spell Checker
        


In [56]:
def correct(word):
    
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)))
    #print(candidates)
    #print(COUNTS.get)
    #print(max(candidates, key=COUNTS.get))
    return(candidates)
    #return max(candidates, key=COUNTS.get)

In [57]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

Now for `edits1(word)`: the set of candidate words that are one edit away. For example, given `"wird"`, this would include `"weird"` (inserting an `e`) and `"word"` (replacing a `i` with a `o`), and also `"iwrd"` (transposing `w` and `i`; then `known` can be used to filter this out of the set of final candidates). How could we get them?  One way is to *split* the original word in all possible places, each split forming a *pair* of words, `(a, b)`, before and after the place, and at each place, either delete, transpose, replace, or insert a letter:

<table>
  <tr><td> pairs: <td><tt> Ø+wird <td><tt> w+ird <td><tt> wi+rd <td><tt>wir+d<td><tt>wird+Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> pair</i>
  <tr><td> deletions: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Delete first char of b</i>
  <tr><td> transpositions: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Swap first two chars of b
  <tr><td> replacements: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>Replace char at start of b
  <tr><td> insertions: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Insert char between a and b
</table>

In [58]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [59]:
list(map(correct, tokens('acress')))

[{'access', 'acres', 'across', 'actress', 'caress'}]

In [60]:
#word = 'Wird:'
def calculate_candidates(word):
    word = tokens_words(word)
    final_candidate_list =[]
    candidates = correct(tokens_words(word))
    
    # When the word does not exist in big.txt, so get it from spell_errors
    if len(candidates) == 0:
        for key,val in spell_errors_dict.items():
            if word in val:
                final_candidate_list.append(key)
    
    else:
        for candidate in candidates:
            if candidate in spell_errors_dict:
                for val1 in  spell_errors_dict[candidate]:
                    if val1 == word:
                        final_candidate_list.append(candidate)

        if(len(final_candidate_list)) == 0:
            final_candidate_list = list(candidates)
            final_candidate_list = list(set(final_candidate_list))
        else:
            final_candidate_list = list(set(final_candidate_list))

    #final_candidate_list = list(set(final_candidate_list))              
    #print(final_candidate_list)
    return final_candidate_list


In [61]:
def find_probability(final_candidate_list):
    probability_of_w = {}
    
    for candidate in final_candidate_list:
        if candidate in COUNTS1_dict:
            probability_of_w[candidate] = (COUNTS1_dict[candidate]/counts1_sum)
        else:
            probability_of_w[candidate] = (COUNTS[candidate]/counts_sum)

    #print(probability_of_w)
    return probability_of_w

In [62]:
def max_probability_word(probability_of_w):
    return max(probability_of_w.items(), key = operator.itemgetter(1))[0]

In [63]:
final_candidate_list = calculate_candidates('hierachial')
probability_of_w = find_probability(final_candidate_list)
print(max_probability_word(probability_of_w))

hierarchical


Can we make the output prettier than that?

In [64]:
test_df = pd.read_csv('test.csv')
print(test_df.head(5))
test_df.drop('ID', axis=1)
test_list = test_df['WRONG'].tolist()
#print(test_list)

   ID       WRONG
0   0  contenpted
1   1    begining
2   2     problam
3   3      dirven
4   4     exstacy


In [65]:
correct_list =[]
for text in test_list:
    final_candidate_list = calculate_candidates(text)
    probability_of_w = find_probability(final_candidate_list)
    correct_list.append(max_probability_word(probability_of_w))
    
print(correct_list)

['contented', 'beginning', 'problem', 'driven', 'ecstasy', 'juice', 'locally', 'compare', 'pronunciation', 'transportability', 'minuscule', 'independent', 'arranged', 'poetry', 'level', 'basically', 'triangular', 'unexpected', 'standardizing', 'variable', 'further', 'monitoring', 'biscuits', 'available', 'separate', 'necessary', 'definition', 'receipt', 'remind', 'initials', 'magnificent', 'aunt', 'initial', 'the', 'experiences', 'built', 'totally', 'understand', 'southern', 'definitely', 'visited', 'voluntary', 'meant', 'receive', 'sources', 'whether', 'useful', 'literature', 'valuable', 'delicate', 'clerical', 'splendid', 'between', 'completely', 'account', 'cemetery', 'special', 'latest', 'perhaps', 'member', 'chapter', 'cake', 'various', 'february', 'pretend', 'choosing', 'wrote', 'particular', 'awful', 'arrangement', 'challenges', 'laugh', 'often', 'someone', 'personnel', 'unique', 'diagrammatically', 'description', 'poems', 'purple', 'decide', 'articles', 'position', 'extended', 

In [66]:
import csv

csvfile = "sample_test_submit4.csv"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in correct_list:
        #print(val)
        writer.writerow([val])  

References:

https://web.stanford.edu/class/cs276/pa/pa2.p

http://web.stanford.edu/~jurafsky/slp3/5.pdf

http://norvig.com/ngrams/spell-errors.txt

http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb

http://norvig.com/big.txt

http://norvig.com/ngrams/count_1w.txt

http://norvig.com/ngrams/